# 5章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
import time

In [2]:
%%time
# 5-1
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
     |████████████████████████████████| 486 kB 43.3 MB/s 
     |████████████████████████████████| 13.4 MB 186 kB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=3aa0d7da8d3e60ad5439f475b1c092259266b2b3882da163938671925c406404
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic
CPU times: user 163 ms, sys: 39.6 ms, total: 202 ms
Wall time: 14.3 s


In [3]:
%%time
# 5-2
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

CPU times: user 850 ms, sys: 752 ms, total: 1.6 s
Wall time: 25.1 s


In [4]:
%%time
# 5-3
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 16.5 s, sys: 4.17 s, total: 20.7 s
Wall time: 40.8 s


In [5]:
%%time
# 5-4
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']
CPU times: user 397 µs, sys: 110 µs, total: 507 µs
Wall time: 516 µs


In [6]:
%%time
# 5-5
# 文章を符号化し、GPUに配置する。
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る。
# 系列長を揃える必要がないので、単にiput_idsのみを入力します。
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

CPU times: user 22.9 ms, sys: 4.71 ms, total: 27.6 ms
Wall time: 233 ms


In [7]:
%%time
torch.cuda.manual_seed(42)
# 5-6
# ID列で'[MASK]'(IDは4)の位置を調べる
mask_position = input_ids[0].tolist().index(4) 

# スコアが最も良いトークンのIDを取り出し、トークンに変換する。
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

# [MASK]を上で求めたトークンで置き換える。
text = text.replace('[MASK]',token_best)

print(text)

今日は東京へ行く。
CPU times: user 2.06 ms, sys: 5 µs, total: 2.06 ms
Wall time: 11.1 ms


In [8]:
# 5-7
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    上位何位まで使うかは、num_topkで指定。
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める。
    mask_position = input_ids[0].tolist().index(4) 
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [9]:
# 5-8
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する。
    """
    # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える。
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [10]:
# 5-9
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は社会社会的な地位'

In [11]:
# 5-10
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [12]:
# 5-11
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。


In [15]:
# 5-12
text = '最後まで嘘ついてまで[MASK]になろうとするんだね'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

最後まで嘘ついてまで幸せになろうとするんだね
最後まで嘘ついてまで金持ちになろうとするんだね
最後まで嘘ついてまで好きになろうとするんだね
最後まで嘘ついてまで大人になろうとするんだね
最後まで嘘ついてまで自由になろうとするんだね
最後まで嘘ついてまで人間になろうとするんだね
最後まで嘘ついてまで幸福になろうとするんだね
最後まで嘘ついてまで友達になろうとするんだね
最後まで嘘ついてまで有名人になろうとするんだね
最後まで嘘ついてまで金になろうとするんだね


In [16]:
# 5-13
text = '君は冬の夢を見てなく[MASK]。もしその願いが[MASK]と知ったら七日目を生きたのかい。'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

君は冬の夢を見てなくなる。もしその願いがあると知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いが正しいと知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いが叶と知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いが来ると知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いがかなと知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いがあると知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いが正しいと知ったら七日目を生きたのかい。
君は冬の夢を見てなくて。もしその願いがあると知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いが本当と知ったら七日目を生きたのかい。
君は冬の夢を見てなくなる。もしその願いがないと知ったら七日目を生きたのかい。


In [18]:
# 5-14
text = '理想と闘争心の[MASK]を締め引き寄せてく'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

理想と闘争心の間を締め引き寄せてく
理想と闘争心の両方を締め引き寄せてく
理想と闘争心の境界を締め引き寄せてく
理想と闘争心の狭間を締め引き寄せてく
理想と闘争心の[UNK]を締め引き寄せてく
理想と闘争心の芽を締め引き寄せてく
理想と闘争心の対立を締め引き寄せてく
理想と闘争心の双方を締め引き寄せてく
理想と闘争心の両者を締め引き寄せてく
理想と闘争心の心を締め引き寄せてく


In [20]:
# 5-15
text = '表に見えたものが実は[MASK]の[MASK]だと気付かない奴らが今夜も街で[MASK]と勇気と投げやりと潔さ履き違えてる。'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

表に見えたものが実は自分のものだと気付かない奴らが今夜も街で勇気と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は裏のものだと気付かない奴らが今夜も街で勇気と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は他人のものだと気付かない奴らが今夜も街で勇気と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は自分のものだと気付かない奴らが今夜も街で[UNK]と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は自分のものだと気付かない奴らが今夜も街で見ると勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は自分のものだと気付かない奴らが今夜も街で堂々と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は本当のものだと気付かない奴らが今夜も街で勇気と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は自分のものだと気付かない奴らが今夜も街で笑顔と勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は裏のものだと気付かない奴らが今夜も街で見ると勇気と投げやりと潔さ履き違えてる。
表に見えたものが実は裏のものだと気付かない奴らが今夜も街で[UNK]と勇気と投げやりと潔さ履き違えてる。
